In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import optuna

In [2]:
Ridership_df= pd.read_csv('data/Ridership.csv')
df= Ridership_df.copy()
df.sample(5)

,Year,Month,Day,Week Number,Corridor,Workday,Station,Period,Ridership,N_trains,Covid19
3646,2019,March,8,10,Corridor_7,y,Station_22,PM Peak,74,2,0
51995,2022,March,30,13,Corridor_7,y,Station_21,AM Peak,778,4,0
42055,2021,September,1,35,Corridor_4,y,Station_8,AM Peak,249,2,1
26843,2020,August,5,32,Corridor_2,y,Station_3,AM Peak,103,3,1
5011,2019,April,8,15,Corridor_3,y,Station_4,PM Peak,965,6,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64369 entries, 0 to 64368
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         64369 non-null  int64 
 1   Month        64369 non-null  object
 2   Day          64369 non-null  int64 
 3   Week Number  64369 non-null  int64 
 4   Corridor     64369 non-null  object
 5   Workday      64369 non-null  object
 6   Station      64369 non-null  object
 7   Period       64369 non-null  object
 8   Ridership    64369 non-null  int64 
 9   N_trains     64369 non-null  int64 
 10  Covid19      64369 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 5.4+ MB


In [4]:
df.duplicated().sum()

np.int64(0)

In [5]:
df.isnull().sum()

Year           0
Month          0
Day            0
Week Number    0
Corridor       0
Workday        0
Station        0
Period         0
Ridership      0
N_trains       0
Covid19        0
dtype: int64

In [6]:
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}
df['Month_Num'] = df['Month'].map(month_mapping)

# Cyclical Day conversion
def convert_day_to_circle(day):
    angle = 2 * np.pi * (day - 1) / 31
    x = np.cos(angle)
    y = np.sin(angle)
    return x, y

df['day_x'], df['day_y'] = zip(*df['Day'].map(convert_day_to_circle))

# Cyclical Week conversion
def convert_week_to_circle(week):
    angle = 2 * np.pi * (week - 1) / 53
    x = np.cos(angle)
    y = np.sin(angle)
    return x, y

df['week_x'], df['week_y'] = zip(*df['Week Number'].map(convert_week_to_circle))


# For Workday, instead of LabelEncoder then OneHotEncoder, let's just let OneHotEncoder handle it directly
categorical_features = ['Month', 'Corridor', 'Workday', 'Station', 'Period']

numerical_features_to_pass_through = [
    'Year', 'N_trains', 'Covid19', 'Ridership',
    'Month_Num', 'day_x', 'day_y', 'week_x', 'week_y']



preprocessor = ColumnTransformer(
    transformers=[
        ('', OneHotEncoder(), categorical_features),
    ],remainder='passthrough')


df = preprocessor.fit_transform(df)
df = df.toarray()
feature_names_out = list(preprocessor.get_feature_names_out())
df = pd.DataFrame(df, columns=[item.split('__')[1] for item in feature_names_out])

df['COVID_Workday'] = df['Covid19'] * df['Workday_y']

In [7]:
columns_to_convert = [
    'Month_April', 'Month_August', 'Month_December', 'Month_February', 
    'Month_January', 'Month_July', 'Month_June', 'Month_March', 
    'Month_May', 'Month_November', 'Month_October', 'Month_September',
    'Corridor_Corridor_1', 'Corridor_Corridor_2', 'Corridor_Corridor_3', 
    'Corridor_Corridor_4', 'Corridor_Corridor_5', 'Corridor_Corridor_6', 'Corridor_Corridor_7',
    'Station_Station_1', 'Station_Station_10', 'Station_Station_11', 'Station_Station_12', 
    'Station_Station_13', 'Station_Station_14', 'Station_Station_15', 'Station_Station_16', 
    'Station_Station_17', 'Station_Station_18', 'Station_Station_19', 'Station_Station_2', 
    'Station_Station_20', 'Station_Station_21', 'Station_Station_22', 'Station_Station_23', 
    'Station_Station_24', 'Station_Station_25', 'Station_Station_26', 'Station_Station_27', 
    'Station_Station_28', 'Station_Station_29', 'Station_Station_3', 'Station_Station_30', 
    'Station_Station_31', 'Station_Station_32', 'Station_Station_33', 'Station_Station_34', 
    'Station_Station_35', 'Station_Station_36', 'Station_Station_37', 'Station_Station_38', 
    'Station_Station_39', 'Station_Station_4', 'Station_Station_40', 'Station_Station_41', 
    'Station_Station_42', 'Station_Station_43', 'Station_Station_44', 'Station_Station_45', 
    'Station_Station_5', 'Station_Station_6', 'Station_Station_7', 'Station_Station_8', 'Station_Station_9',
    'Period_AM Peak', 'Period_Evening', 'Period_Midday', 'Period_PM Peak', 'Period_Weekend/Holiday',
    'Day',
    'Week Number',
    'Workday_n', 'Workday_y',
    'Covid19','COVID_Workday',
    'Month_Num']

columns_to_convert_02=['Year', 'Ridership', 'N_trains']


df[columns_to_convert] = df[columns_to_convert].astype('uint8')
df[columns_to_convert_02] = df[columns_to_convert_02].astype('uint16')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64369 entries, 0 to 64368
Data columns (total 83 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Month_April             64369 non-null  uint8  
 1   Month_August            64369 non-null  uint8  
 2   Month_December          64369 non-null  uint8  
 3   Month_February          64369 non-null  uint8  
 4   Month_January           64369 non-null  uint8  
 5   Month_July              64369 non-null  uint8  
 6   Month_June              64369 non-null  uint8  
 7   Month_March             64369 non-null  uint8  
 8   Month_May               64369 non-null  uint8  
 9   Month_November          64369 non-null  uint8  
 10  Month_October           64369 non-null  uint8  
 11  Month_September         64369 non-null  uint8  
 12  Corridor_Corridor_1     64369 non-null  uint8  
 13  Corridor_Corridor_2     64369 non-null  uint8  
 14  Corridor_Corridor_3     64369 non-null

# spliting

In [9]:
X = df.drop(['Ridership', 'Year', 'N_trains'], axis=1)  
y = pd.DataFrame(df['Ridership'])

df_for_stratify = Ridership_df.copy()
stratify_key = df_for_stratify['Covid19'].astype(str) + '_' + \
               df_for_stratify['Workday'] + '_' + \
               df_for_stratify['Period'] + '_' + \
               df_for_stratify['Corridor']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=stratify_key 
)

## Problem One: 
Assume we want to find a general model for the number of passengers without looking at the number of passengers on previous days and only using the recorded information in the table for the same time period (except for the year and number of trains) to predict the number of required trains based on the number of passengers.

In [26]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Define parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'verbose': -1
}

# Train the model
model_p01 = lgb.train(params,
                 train_data,
                 num_boost_round=1000,
                 valid_sets=[test_data],
                 callbacks=[lgb.early_stopping(stopping_rounds=50)])

# Make predictions
y_pred = model_p01.predict(X_test)

# Evaluate
rmse_f_p01 = np.sqrt(mean_squared_error(y_test, y_pred))
r2_f_p01= r2_score(y_test, y_pred)

print(f"RMSE: {rmse_f_p01:.2f}")
print(f"R²: {r2_f_p01:.4f}")

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 825.099
RMSE: 825.10
R²: 0.8129


In [10]:

def objective(trial):
    # Define parameters
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'verbose': -1,
        'feature_pre_filter': False  # Disable feature pre-filtering
    }

    # Convert data to LightGBM Dataset format inside the objective function
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

    # Train the model
    model = lgb.train(params,
                      train_data,
                      num_boost_round=1000,
                      valid_sets=[test_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=50)])
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Return RMSE as the objective value to minimize
    return np.sqrt(mean_squared_error(y_test, y_pred))

# Create a study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Best parameters
best_params = study.best_params
print("Best parameters:", best_params)


[I 2025-07-21 02:56:42,652] A new study created in memory with name: no-name-72727980-4bb5-4e57-b266-a3b90fab66a9


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[655]	valid_0's rmse: 983.857


[I 2025-07-21 02:56:44,671] Trial 0 finished with value: 983.8571349781687 and parameters: {'num_leaves': 58, 'learning_rate': 0.2661990120612658, 'min_child_samples': 56, 'max_depth': 4, 'feature_fraction': 0.8360333006471041, 'bagging_fraction': 0.5103394541520616, 'bagging_freq': 6, 'lambda_l1': 5.591061258626268, 'lambda_l2': 8.857246617297712}. Best is trial 0 with value: 983.8571349781687.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 1043.23


[I 2025-07-21 02:56:46,896] Trial 1 finished with value: 1043.2277685454872 and parameters: {'num_leaves': 81, 'learning_rate': 0.057583280963296644, 'min_child_samples': 99, 'max_depth': 4, 'feature_fraction': 0.5713756717175975, 'bagging_fraction': 0.7716852580905575, 'bagging_freq': 1, 'lambda_l1': 3.341962948158439, 'lambda_l2': 7.293504490264829}. Best is trial 0 with value: 983.8571349781687.


...

Early stopping, best iteration is:
[152]	valid_0's rmse: 846.152
Training until validation scores don't improve for 50 rounds


[I 2025-07-21 02:58:42,138] Trial 48 finished with value: 847.6034507757207 and parameters: {'num_leaves': 78, 'learning_rate': 0.2053498891104058, 'min_child_samples': 9, 'max_depth': 12, 'feature_fraction': 0.9896234768875556, 'bagging_fraction': 0.7470685377893002, 'bagging_freq': 1, 'lambda_l1': 9.661167163936657, 'lambda_l2': 1.1833875118496695}. Best is trial 46 with value: 826.0919618221408.


Early stopping, best iteration is:
[139]	valid_0's rmse: 847.603
Training until validation scores don't improve for 50 rounds


[I 2025-07-21 02:58:43,309] Trial 49 finished with value: 826.8852339347029 and parameters: {'num_leaves': 82, 'learning_rate': 0.14988409244644552, 'min_child_samples': 5, 'max_depth': 12, 'feature_fraction': 0.9054527736795613, 'bagging_fraction': 0.8666852990946773, 'bagging_freq': 4, 'lambda_l1': 9.21690540293889, 'lambda_l2': 0.8250564187001804}. Best is trial 46 with value: 826.0919618221408.


Early stopping, best iteration is:
[201]	valid_0's rmse: 826.885
Best parameters: {'num_leaves': 89, 'learning_rate': 0.20404032634999653, 'min_child_samples': 5, 'max_depth': 12, 'feature_fraction': 0.9111908365749342, 'bagging_fraction': 0.8685835230778026, 'bagging_freq': 2, 'lambda_l1': 9.119218773197982, 'lambda_l2': 0.7001947797129606}


In [19]:
# Train final model with all data and best parameters
lgb_model_p01 = lgb.train(best_params,
                       lgb.Dataset(pd.concat([X_train, X_test]), 
                                 label=pd.concat([y_train, y_test])),
                       num_boost_round=1000)

y_pred_01 = lgb_model_p01.predict(X_test)

# Evaluate
rmse_p01 = np.sqrt(mean_squared_error(y_test, y_pred_01 ))
r2_p01 = r2_score(y_test, y_pred_01 )

print(f"RMSE: {rmse_p01 :.2f}")
print(f"R²: {r2_p01 :.4f}")

RMSE: 508.88
R²: 0.9288


In [24]:
y_pred_train_p01 = pd.DataFrame(y_pred_01, columns=['Predicted_Passengers']).astype(int)

train_capacity = 600  
y_pred_train_p01['Predicted_Trains'] = np.ceil(y_pred_train_p01['Predicted_Passengers'] / train_capacity).astype(int)

# Add a Safety/Comfort Buffer (e.g., 5% more capacity)
# This allocates slightly more trains than the absolute minimum required.
safety_buffer_percentage = 0.05 
y_pred_train_p01['Buffered_Predicted_Trains'] = np.ceil(
    y_pred_train_p01['Predicted_Passengers'] * (1 + safety_buffer_percentage) / train_capacity).astype(int)

# 4. Ensure a Minimum Number of Trains (e.g., at least 1 train, even for 0 passengers)
minimum_trains_required = 1
y_pred_train_p01['Final_Predicted_Trains'] = y_pred_train_p01['Buffered_Predicted_Trains'].apply(
    lambda x: max(x, minimum_trains_required))

y_pred_train_p01.sample(10)

,Predicted_Passengers,Predicted_Trains,Buffered_Predicted_Trains,Final_Predicted_Trains
11431,184,1,1,1
6038,207,1,1,1
1118,5362,9,10,10
9304,-162,0,0,1
1367,340,1,1,1
12594,281,1,1,1
6748,238,1,1,1
8771,55,1,1,1
5212,1072,2,2,2
149,1745,3,4,4
